In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import matplotlib.pyplot as plt # for plotting
import torch.optim as optim #for gradient descent
import torchvision
#import torchvision.transforms as transforms
#from torch.utils.data.sampler import SubsetRandomSampler
import numpy as np

torch.manual_seed(1) # set the random seed
print(torch.__version__, torch.cuda.is_available())

use_cuda = True

2.0.0+cu117 True


In [2]:
import torch
import os
import zipfile
import numpy as np
import pandas as pd 
from torchvision.datasets import DatasetFolder
from torch.utils import data
from lightning import LightningDataModule
from torchvision import transforms
from torch.utils.data import Dataset, ConcatDataset
from imblearn.over_sampling import RandomOverSampler 
from imblearn.under_sampling import RandomUnderSampler
from PIL import Image
from concurrent.futures import ThreadPoolExecutor
class ImageDataset(Dataset):
    def __init__(self, dir, transform=None, load_first=True, testing=True, new_data=False):
        self.base_dir = dir
        self.nih_dir = dir + r"\data\nih_data"
        self.pneumonia_dir = dir + r"\data\pneumonia"
        self.pneumothorax_dir = dir + r"\data\pneumothorax"
        self.transform = transform
        self.load_first = load_first
        self.testing = testing
        self.new_data = new_data

        self.df = self._inital_processing(dir)
        
        if new_data:
            self.labels = torch.FloatTensor(self.df.iloc[:, 1:].values.astype(int)).float()
        else:
            self.labels = torch.FloatTensor(np.array(self.df.iloc[:, 4:]).astype(int)).float()

        if load_first:
            self.images = self._load_all_images_parallel()

        print(self.nih_dir)
        # print(self.labels)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, index):
        if self.load_first:
            # obtain image from self.images
            image = self.images[index]
            img_path = self.base_dir + fr"\{self.df.iloc[index, 0]}"
        else:
            if not self.new_data:
                if self.df.iloc[index,0] == 1:
                    img_path = self.nih_dir + fr"\{self.df.iloc[index, 1]}"

                #For Pneumothorax Images
                elif self.df.iloc[index,0] == 2:
                    img_path = self.pneumothorax_dir + fr"\{self.df.iloc[index, 1]}"

                #For Pneumonia Images
                elif self.df.iloc[index,0] == 3:
                    img_path = self.pneumonia_dir + fr"\{self.df.iloc[index, 1]}"
            else:
                img_path = os.path.join(self.base_dir, self.df.iloc[index, 0])
                print(img_path)
            # open then close the image to avoid too many open files error
            with Image.open(img_path) as image:
                image.load()

        if self.transform is not None:
            image = self.transform(image)

        label = self.labels[index]

        if self.testing:
            return image, label, index, img_path
        return image, label
    
    def _inital_processing(self, dir):
        file_name = "Allimages_onehot.csv" if not self.new_data else "newdata_onehot.csv"
        dataset_df = pd.read_csv(os.path.join(dir, "csv_mappings", file_name))

        if not self.new_data:
            # subset only nih data for now by column name
            # where only num column == 1
            # dataset_df = dataset_df[dataset_df['Num'] == 1]

            # undersample the no findings examples
            # they are the ones with all 0's as labels
            # randomly select 5000 of them
            no_findings = dataset_df[dataset_df.iloc[:,4:].sum(axis=1) == 0]
            no_findings = no_findings.sample(n=30000, random_state=26)
            dataset_df = dataset_df[dataset_df.iloc[:,4:].sum(axis=1) != 0]
            dataset_df = pd.concat([dataset_df, no_findings])

        return dataset_df

    def _load_all_images(self):
        images = []
        for index in range(len(self.df)):
            img_path = self.dir + self.df.iloc[index, 0]
            # open then close the image to avoid too many open files error
            with open(img_path, 'rb') as image:
                image = Image.open(image)
                image.load()
                images.append(image.convert('RGB'))
        return images
    
    def _load_image(self, start_index, end_index):
        images = []
        for index in range(start_index, end_index):
            if not self.new_data:
                if self.df.iloc[index,0] == 1:
                    img_path = self.nih_dir + fr"\{self.df.iloc[index, 1]}"
                elif self.df.iloc[index,0] == 2:
                    img_path = self.pneumothorax_dir + fr"\{self.df.iloc[index, 1]}"
                elif self.df.iloc[index,0] == 3:
                    img_path = self.pneumonia_dir + fr"\{self.df.iloc[index, 1]}"
            else: 
                print("File path:", self.df.iloc[index, 0])
                img_path = self.base_dir +  fr"\{self.df.iloc[index, 0]}"
            print(img_path)
            # open then close the image to avoid too many open files error
            with Image.open(img_path) as image:
                image.load()
                images.append(image)
        return images

    def _load_all_images_parallel(self):
        # load the images in parallel
        # maintain the order of the images
        # return a list of images
        images = []
        with ThreadPoolExecutor() as executor:
            # partition job into chunks
            chunk_size = 1000
            num_chunks = len(self.labels) // chunk_size
            for i in range(num_chunks):
                start_index = i * chunk_size
                end_index = start_index + chunk_size
                images += executor.submit(self._load_image, start_index, end_index).result()
            # process the remaining images
            start_index = num_chunks * chunk_size
            end_index = len(self.labels)
            images += executor.submit(self._load_image, start_index, end_index).result()

        return images

class LungDetectionDataModule(LightningDataModule):

    def __init__(self, batch_size=2, num_workers=0, master_path="", load_first = True, testing = False, new_data = False):
        super().__init__()
        self.batch_size = batch_size
        self.num_workers = num_workers

        self.data_dir = master_path

        initial_tranforms = [
            transforms.Lambda(lambda x: x.convert('RGB')),
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor()
        ]
        datatransform = transforms.Compose(initial_tranforms)
        dataset = self._create_dataset(master_path, datatransform, load_first, testing, new_data)
        train_len, val_len = int(len(dataset)*0.8), int(len(dataset)*0.1)
        test_len = len(dataset) - train_len - val_len
        train, valid, test = torch.utils.data.random_split(
            dataset,
            [train_len, val_len, test_len],
        )
        print("train: ", len(train), "valid: ", len(valid), "test: ", len(test))
        self.train = train
        self.valid = valid
        self.test = test

        mean = torch.tensor([0.485, 0.456, 0.406])
        std = torch.tensor([0.229, 0.224, 0.225])

        non_train_transforms_list = [
            transforms.RandomHorizontalFlip(),
            transforms.AugMix(severity=6, mixture_width=6),
            transforms.TrivialAugmentWide(),
            transforms.Normalize(mean=mean, std=std),
        ]
        non_train_transforms = transforms.Compose(non_train_transforms_list)
        self.valid.transform = non_train_transforms
        self.test.transform = non_train_transforms

        train_transforms_list = [
            transforms.RandomHorizontalFlip(),
            transforms.AugMix(severity=6, mixture_width=6),
            transforms.TrivialAugmentWide(),
            transforms.Normalize(mean=mean, std=std),
        ]
        train_transform = transforms.Compose(train_transforms_list)

        self.train.transform = train_transform
    
    def _create_dataset(self, path, transforms = None, load_first = True, testing = False, new_data = False): 
        print('new_data: ', new_data)
        return ImageDataset(path, transforms, load_first, testing, new_data)
    
    def train_dataloader(self):
        return data.DataLoader(
            self.train,
            batch_size=self.batch_size,
            shuffle=True,
            num_workers=self.num_workers,
            pin_memory=True
        )

    def val_dataloader(self):
        return data.DataLoader(
            self.valid,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=self.num_workers,
            pin_memory=True
        )

    def test_dataloader(self):
        return data.DataLoader(
            self.test,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=self.num_workers,
            pin_memory=True
        )


# Defining and Training the Model

In [3]:
#defining the model - CNN with one convolution, make it into it
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.conv1 = nn.Conv2d(1024, 1048, 3, 1, 1) #input is over 1000
        self.pool = nn.MaxPool2d(2, 2)
        self.act1 = nn.LeakyReLU()
        self.act2 = nn.LeakyReLU()
        self.act3 = nn.PReLU()
        self.act4 = nn.PReLU()
        self.act5 = nn.PReLU()
        self.act6 = nn.PReLU()
        self.fc1 = nn.Linear(3*3*1048, 2048)
        self.fc2 = nn.Linear(2048, 1024)
        self.fc3 = nn.Linear(1024, 512)
        self.fc4 = nn.Linear(512, 256)
        self.fc5 = nn.Linear(256, 96)
        self.out = nn.Linear(96, 14) 

    def forward(self, x):
        x = self.pool(self.act1(self.conv1(x)))
        x = x.view(-1, 3*3*1048)
        x = self.act2(self.fc1(x))
        x = self.act3(self.fc2(x))
        x = self.act4(self.fc3(x))
        x = self.act5(self.fc4(x))
        x = self.act6(self.fc5(x))
        return self.out(x)

model = Classifier()
model = model.cuda()

In [4]:
from torchvision import models

class BaselinePredictor(nn.Module):
    def __init__(self, fine_tune=False, train_from_scratch = False, **kwargs):
        super().__init__()
        self.baseline_model = Classifier()

        if not train_from_scratch:
            if fine_tune:
                self.dense_model = self._init_dense_model(train_from_scratch)
                # don't initially allow the dense model to be trained
                for param in self.dense_model.parameters():
                    param.requires_grad = False
            else:
                # create dense net model but freeze the parameters
                self.dense_model = self._init_dense_model(train_from_scratch)
                for param in self.dense_model.parameters():
                    param.requires_grad = False
        else:
            self.dense_model = self._init_dense_model(train_from_scratch)

    def forward(self, input_embeds):
        if hasattr(self, 'dense_model'):
            input_embeds = self.dense_model(input_embeds)
        return self.baseline_model(input_embeds)

    def _init_dense_model(self, train_from_scratch):
        if not train_from_scratch:
            checkpoint = torch.load('../data_processing/model.pth.tar',
                                    map_location=torch.device('cuda:0'))
            #loading the dictionary of the checkpoint and loading the densent model
            dense_model = models.densenet121(pretrained=True, drop_rate=0.3).cuda()
            model_dict = dense_model.state_dict()
            saved_state_dict = checkpoint['state_dict']

            # Modify the keys in the saved state dict to match the keys in your model
            newdict = {}
            for key, value in saved_state_dict.items():
                new_key = key.replace('densenet121.', '')
                new_key = new_key.replace('norm.', 'norm')
                new_key = new_key.replace('conv.', 'conv')
                new_key = new_key.replace('normr', 'norm.r')
                new_key = new_key.replace('normb', 'norm.b')
                new_key = new_key.replace('normw', 'norm.w')
                new_key = new_key.replace('convw', 'conv.w')
                newdict[new_key] = value

            #ignoring the model checkpoint's classifiers
            model_dict = dense_model.state_dict()
            checkpoint_dict = {k: v for k, v in newdict.items() if k in model_dict}
            model_dict.update(checkpoint_dict)
            #loading in the model dictionary
            dense_model.load_state_dict(model_dict)
        else: 
            dense_model = models.densenet121(pretrained=False).cuda()
        return dense_model.features

In [5]:
import torchmetrics
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch import nn
from torch.optim.lr_scheduler import CosineAnnealingLR, ReduceLROnPlateau
from lightning import LightningModule
from adamp import AdamP

class BaselineLightning(LightningModule):
    def __init__(self,
                 lr,
                 momentum,
                 weight_decay,
                 gamma=2,
                 alpha=None,
                 pos_weight_vec = None,
                 fine_tune_epoch_start= 20, 
                 **kwargs):
        super().__init__()
        self.save_hyperparameters()
        self.model = BaselinePredictor(**kwargs)
        self.lr = lr
        self.num_classes = kwargs['num_classes']
        self.momentum = momentum
        self.wd = weight_decay
        self.criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight_vec)
        self.fine_tune_epoch_start = fine_tune_epoch_start
        self.is_fine_tuning = False
        self.disease_mapping = {
            0: "Atelectasis",
            1: "Cardiomegaly",
            2: "Consolidation",
            3: "Edema",
            4: "Effusion",
            5: "Emphysema",
            6: "Fibrosis",
            7: "Hernia",
            8: "Infiltration",
            9: "Mass",
            10: "Nodule",
            11: "Pleural_Thickening",
            12: "Pneumonia",
            13: "Pneumothorax",
        }

        self.test_results = []

    def forward(self, chexnet_embeds):
        return self.model(chexnet_embeds)

    def training_step(self, batch, batch_idx):
        # unfreeze dense net after epoch
        if self.current_epoch >= self.fine_tune_epoch_start-1 and not self.is_fine_tuning:
            print(f"Unfreezing DenseNet after {self.fine_tune_epoch_start} epochs")
            for param in self.model.dense_model.parameters():
                param.requires_grad = True
            self.is_fine_tuning = True

        loss, batch_size, train_accuracy, f1_score, f1_score_avg, weighted_f1, auroc = self._process_batch(
            batch, True)
        self.log('train_loss', loss, batch_size=batch_size, prog_bar=True)
        self.log('train_accuracy',
                 train_accuracy,
                 batch_size=batch_size,
                 prog_bar=True)
        self.log('train_f1_score', f1_score_avg, batch_size=batch_size)
        self.log('train_weighted_f1', weighted_f1, batch_size=batch_size)
        self._log_f1_score(f1_score, "train", batch_size)
        self._log_auroc(auroc, "train", batch_size)

        return loss

    def validation_step(self, batch, batch_idx):
        loss, batch_size, val_accuracy, f1_score, f1_score_avg, weighted_f1, auroc = self._process_batch(
            batch, True)
        self.log('val_loss',
                 loss,
                 batch_size=batch_size,
                 on_step=False,
                 on_epoch=True,
                 prog_bar=True)
        self.log('val_accuracy',
                 val_accuracy,
                 batch_size=batch_size,
                 on_step=False,
                 on_epoch=True,
                 prog_bar=True)
        self.log('val_f1_score',
                 f1_score_avg,
                 batch_size=batch_size,
                 on_step=False,
                 on_epoch=True,
                 prog_bar=True)
        self.log('val_weighted_f1',
                 weighted_f1,
                 batch_size=batch_size,
                 on_step=False,
                 on_epoch=True,
                 prog_bar=True)
        self._log_f1_score(f1_score, "val", batch_size)
        self._log_auroc(auroc, "val", batch_size)

        return loss

    def test_step(self, batch, batch_idx):
        loss, batch_size, test_accuracy, f1_score, f1_score_avg, weighted_f1, auroc, logits, labels, index, img_path = self._process_batch(
            batch, True, True)
        self.log('test_loss', loss, batch_size=batch_size, on_epoch=True)
        self.log('test_accuracy', test_accuracy, batch_size=batch_size, on_epoch=True)
        self.log('test_f1_score', f1_score_avg, batch_size=batch_size, on_epoch=True)
        self.log('test_weighted_f1', weighted_f1, batch_size=batch_size, on_epoch=True)
        self._log_f1_score(f1_score, "test", batch_size, on_epoch=True)
        self._log_auroc(auroc, "test", batch_size, on_epoch=True)

        probs = torch.sigmoid(logits).float().cpu().detach().numpy()
        preds = (probs > 0.5).astype(int)
        labels = labels.cpu().detach().numpy()
        is_correct = (preds == labels).all(axis=1)

        self.test_results.append({
            "probs": probs,
            "prediction": preds,
            "label": labels,
            "dataframe_index": index,
            "img_path": img_path,
            "is_correct": is_correct
        })

        return loss

    def configure_optimizers(self):
        # optimizer = torch.optim.SGD(
        #     self.parameters(), lr=self.lr,
        #     weight_decay=self.wd, momentum=self.momentum, nesterov=True)
        # optimizer = SGDP(self.parameters(),
        #                 lr=self.lr,
        #                 weight_decay=self.wd,
        #                 momentum=self.momentum,
        #                 nesterov=True)
        
        # optimizer = torch.optim.Adam(self.parameters(),
        #                              lr=self.lr,
        #                              weight_decay=self.wd)
        optimizer = AdamP(self.parameters(), 
                          lr=self.lr, 
                          weight_decay=self.wd,
                          nesterov=True)
        
        cosine_anneal = CosineAnnealingLR(optimizer, T_max=6, eta_min=0.00015)
        reduce_lr = ReduceLROnPlateau(optimizer, min_lr=0.00001)

        return {
            "optimizer": optimizer,
            "lr_scheduler": cosine_anneal,
            "monitor": "val_loss"
        }

    def _process_batch(self, batch, compute_accuracy=False, return_logits = False):
        if return_logits:
            chexnet_embeds, labels, index, img_path = batch
        else:
            chexnet_embeds, labels = batch
        logits = self(chexnet_embeds)
        loss = self.criterion(logits, labels)
        if compute_accuracy:
            labels = labels.int()
            num_labels = int(self.num_classes)

            # debug code
            #print(F.sigmoid(logits).shape, labels.shape)
            # convert logit prob to binary
            #bin_preds = torch.round(F.sigmoid(logits)).int()
            #print("Preds: ", bin_preds, "labels:", labels)

            accuracy_metric = torchmetrics.Accuracy(task="multilabel",
                                                    num_labels=num_labels,
                                                    average="macro").cuda()
            accuracy = accuracy_metric(logits, labels)
            # pytorch f1 score
            f1_score_metric = torchmetrics.F1Score(task="multilabel",
                                                   num_labels=num_labels,
                                                   average='none').cuda()
            f1_score = f1_score_metric(logits, labels)
            # f1_score = multiclass_f1_score(logits, labels, average=None, num_classes=self.num_classes)
            f1_score_avg = f1_score.mean()

            weighted_f1_score_metric = torchmetrics.F1Score(
                task="multilabel", num_labels=num_labels,
                average='weighted').cuda()
            weighted_f1_score = weighted_f1_score_metric(logits, labels)

            # pytorch auroc scores per class
            auroc_metric = torchmetrics.AUROC(task="multilabel",
                                              num_labels=num_labels,
                                              average="none").cuda()
            auroc = auroc_metric(logits, labels)
            # auroc = multiclass_auroc(logits, labels, average=None, num_classes=self.num_classes)
            if not return_logits:
                return loss, len(
                    labels
                ), accuracy, f1_score, f1_score_avg, weighted_f1_score, auroc
            else:
                return loss, len(
                    labels
                ), accuracy, f1_score, f1_score_avg, weighted_f1_score, auroc, logits, labels, index, img_path

        return loss, len(labels)

    def _log_f1_score(self, f1_score, step_type, batch_size, on_epoch=True):
        for i in range(len(f1_score)):
            self.log(f"{step_type}_f1_score_class_{self.disease_mapping[i]}",
                     f1_score[i],
                     batch_size=batch_size, on_epoch=on_epoch)

    def _log_auroc(self, auroc, step_type, batch_size, on_epoch=True):
        for i in range(len(auroc)):
            self.log(f"{step_type}_auroc_class_{self.disease_mapping[i]}",
                     auroc[i],
                     batch_size=batch_size, on_epoch=on_epoch)


In [6]:
import wandb
import torch
import os
import csv
import numpy as np
import pandas as pd
from lightning.pytorch.loggers import WandbLogger
from lightning.pytorch.callbacks import ModelSummary
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch.callbacks import EarlyStopping
from lightning.pytorch.callbacks import LearningRateMonitor
from lightning.pytorch.callbacks import StochasticWeightAveraging
from lightning.pytorch.trainer import Trainer, seed_everything

def train_main(batch_size=128, num_workers=4, max_epochs=50,
               master_path="", use_inverse_weighting = False,
               pos_weight_multi=1.0, load_first = False, 
               fine_tune_epoch_start=20, checkpoint_path = None, 
               testing = False, new_data = False, **kwargs):
    # seed experiment
    seed_everything(seed=26)

    # construct datamodule
    datamodule = LungDetectionDataModule(batch_size=batch_size,
                                         num_workers=num_workers,
                                         master_path=master_path, 
                                         load_first=load_first, 
                                         testing=testing,
                                         new_data=new_data)
    data_size = len(datamodule.train)
    if use_inverse_weighting:
        targets = [target for _, target in datamodule.train]
        class_counts = np.bincount(targets)

        # Calculate the class frequencies
        class_freqs = class_counts / data_size

        # Calculate the inverse frequency weights
        weights = 1 / class_freqs
    else:
        weights = None

    # ratio of negative examples to positive examples 
    # for bce with logits loss 
    # consider the labels from datamodule.train 
    num_pos_labels_train = torch.sum(datamodule.train.dataset.labels, dim=0)
    num_neg_labels_train = len(datamodule.train) - num_pos_labels_train
    pos_weight_vec = num_neg_labels_train / num_pos_labels_train
    pos_weight_vec = pos_weight_vec * pos_weight_multi

    # construct model
    lit_model = BaselineLightning(seed=123,
                              batch_size=batch_size,
                              num_workers=num_workers,
                              data_size=data_size,
                              alpha=weights,
                              pos_weight_vec=pos_weight_vec,
                              fine_tune_epoch_start=fine_tune_epoch_start,
                              **kwargs)

    # logging
    logger = WandbLogger(project="lung-xray-baseline", entity="ericzhu",
                         log_model="all", save_dir="./wandb_saves")
    logger.experiment.config["train_set_len"] = len(datamodule.train)
    logger.experiment.config["val_set_len"] = len(datamodule.valid)
    logger.experiment.config["batch_size"] = batch_size

    # callbacks
    early_stopping = EarlyStopping(
        monitor="val_f1_score", mode="max", patience=100)
    checkpointing = ModelCheckpoint(
        monitor="val_f1_score", mode="max", save_top_k=5)
    stochastic_weighting = StochasticWeightAveraging(swa_epoch_start=0.75,
                                                     annealing_epochs=8,
                                                     swa_lrs=0.0003)
    model_sumary = ModelSummary(max_depth=4)
    learning_rate_montior = LearningRateMonitor(logging_interval="step")
    # training
    trainer = Trainer(
        callbacks=[early_stopping, checkpointing, model_sumary,
                   learning_rate_montior],
        devices="auto",
        logger=logger,
        enable_progress_bar=True,
        log_every_n_steps=1,
        max_epochs=max_epochs,
        precision="bf16-mixed",
    )
    if not testing:
        trainer.fit(lit_model, datamodule=datamodule, ckpt_path=checkpoint_path)
    else:
        test_results = trainer.test(lit_model, datamodule=datamodule, ckpt_path=checkpoint_path)
        wandb.finish()
        return test_results, lit_model

    wandb.finish()

    return lit_model

In [7]:
torch.set_float32_matmul_precision("medium")
master_path = r'C:\Users\ericz\Documents\Github\APS360\Final Project\data_processing'
train = False
if train:
    train_configs = {
        "master_path": master_path,
        "batch_size": 256,
        "num_workers": 0,
        "max_epochs": 5,
        "lr": 0.006,
        "weight_decay": 8e-8,
        "momentum": 0.98,
        "gamma": 2, 
        "use_inverse_weighting": False,
        "num_classes": 14,
        "fine_tune": False,
        "fine_tune_epoch_start": 40,
        "pos_weight_multi": 0,
        "train_from_scratch": True,
        "load_first": True,
    }
    baseline_model = train_main(**train_configs)
else:
    run = wandb.init(project="lung-xray", entity="ericzhu",)
    artifact = run.use_artifact(
        'ericzhu/lung-xray-baseline/model-8by6l2gg:v0', type='model')
    artifact_dir = artifact.download()
    model_checkpoint = os.path.join(artifact_dir, "model.ckpt")
    train_configs = {
        "master_path": master_path,
        "batch_size": 256,
        "num_workers": 0,
        "max_epochs": 5,
        "lr": 0.006,
        "weight_decay": 8e-8,
        "momentum": 0.98,
        "gamma": 2, 
        "use_inverse_weighting": False,
        "num_classes": 14,
        "fine_tune": True,
        "fine_tune_epoch_start": 40,
        "pos_weight_multi": 0,
        "train_from_scratch": False,
        "load_first": False,
        "checkpoint_path": model_checkpoint,
        "testing": not train,
        "new_data": True,
    }
    test_results, eval_model = train_main(**train_configs)
    print(test_results)

    # save test results to txt file
    with open("test_results.txt", "w") as f:
        f.write(str(test_results))

    test_results = eval_model.test_results
    test_results_df = pd.DataFrame(test_results)
    test_results_df.to_csv("test_predictions.csv")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: ericzhu. Use `wandb login --relogin` to force relogin


wandb: Downloading large artifact model-8by6l2gg:v0, 443.84MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:0.2
Global seed set to 26


C:\Users\ericz\Documents\Github\APS360\Final Project\data_processing\data\nih_data
train:  70960 valid:  8870 test:  8871


c:\Users\ericz\anaconda3\envs\pytorch-latest\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\ericz\anaconda3\envs\pytorch-latest\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
c:\Users\ericz\anaconda3\envs\pytorch-latest\lib\site-packages\lightning\pytorch\loggers\wandb.py:395: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
Us

Testing: 0it [00:00, ?it/s]

c:\Users\ericz\anaconda3\envs\pytorch-latest\lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning: No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score
  warnings.warn(*args, **kwargs)


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃              Test metric               ┃              DataLoader 0              ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│             test_accuracy              │           0.9295457005500793           │
│      test_auroc_class_Atelectasis      │                  0.5                   │
│     test_auroc_class_Cardiomegaly      │                  0.5                   │
│     test_auroc_class_Consolidation     │                  0.5                   │
│         test_auroc_class_Edema         │                  0.5                   │
│       test_auroc_class_Effusion        │                  0.5                   │
│       test_auroc_class_Emphysema       │                  0.5                   │
│       test_auroc_class_Fibrosis        │                  0.5                   │
│        test_auroc_class_Hernia         │           0.2885807752609253           │
│     test_auroc_class_Infiltration      │                  0.5                   │
│         test_auroc_class_Mass          │                  0.5                   │
│        test_auroc_class_Nodule         │                  0.5                   │
│  test_auroc_class_Pleural_Thickening   │                  0.5                   │
│       test_auroc_class_Pneumonia       │                  0.5                   │
│     test_auroc_class_Pneumothorax      │                  0.5                   │
│             test_f1_score              │                  0.0                   │
│    test_f1_score_class_Atelectasis     │                  0.0                   │
│    test_f1_score_class_Cardiomegaly    │                  0.0                   │
│   test_f1_score_class_Consolidation    │                  0.0                   │
│       test_f1_score_class_Edema        │                  0.0                   │
│      test_f1_score_class_Effusion      │                  0.0                   │
│     test_f1_score_class_Emphysema      │                  0.0                   │
│      test_f1_score_class_Fibrosis      │                  0.0                   │
│       test_f1_score_class_Hernia       │                  0.0                   │
│    test_f1_score_class_Infiltration    │                  0.0                   │
│        test_f1_score_class_Mass        │                  0.0                   │
│       test_f1_score_class_Nodule       │                  0.0                   │
│ test_f1_score_class_Pleural_Thickening │                  0.0                   │
│     test_f1_score_class_Pneumonia      │                  0.0                   │
│    test_f1_score_class_Pneumothorax    │                  0.0                   │
│               test_loss                │                  0.0                   │
│            test_weighted_f1            │                  0.0                   │
└────────────────────────────────────────┴────────────────────────────────────────┘

epoch,▁
test_accuracy,▁
test_auroc_class_Atelectasis,▁
test_auroc_class_Cardiomegaly,▁
test_auroc_class_Consolidation,▁
test_auroc_class_Edema,▁
test_auroc_class_Effusion,▁
test_auroc_class_Emphysema,▁
test_auroc_class_Fibrosis,▁
test_auroc_class_Hernia,▁
test_auroc_class_Infiltration,▁


[{'test_loss': 0.0, 'test_accuracy': 0.9295457005500793, 'test_f1_score': 0.0, 'test_weighted_f1': 0.0, 'test_f1_score_class_Atelectasis': 0.0, 'test_f1_score_class_Cardiomegaly': 0.0, 'test_f1_score_class_Consolidation': 0.0, 'test_f1_score_class_Edema': 0.0, 'test_f1_score_class_Effusion': 0.0, 'test_f1_score_class_Emphysema': 0.0, 'test_f1_score_class_Fibrosis': 0.0, 'test_f1_score_class_Hernia': 0.0, 'test_f1_score_class_Infiltration': 0.0, 'test_f1_score_class_Mass': 0.0, 'test_f1_score_class_Nodule': 0.0, 'test_f1_score_class_Pleural_Thickening': 0.0, 'test_f1_score_class_Pneumonia': 0.0, 'test_f1_score_class_Pneumothorax': 0.0, 'test_auroc_class_Atelectasis': 0.5, 'test_auroc_class_Cardiomegaly': 0.5, 'test_auroc_class_Consolidation': 0.5, 'test_auroc_class_Edema': 0.5, 'test_auroc_class_Effusion': 0.5, 'test_auroc_class_Emphysema': 0.5, 'test_auroc_class_Fibrosis': 0.5, 'test_auroc_class_Hernia': 0.2885807752609253, 'test_auroc_class_Infiltration': 0.5, 'test_auroc_class_Mass':